### 03. 단어 단위 RNN - 임베딩 사용
- 문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용
- 그리고 단어 단위를 사용함에 따라서 Pytorch에서 제공하는 임베딩 층 사용

#### 1. 훈련 데이터 전처리

In [1]:
# 필요한 도구 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# 임의의 문장 생성
sentence = "Repeat is the best medicine for memory".split()
print(sentence)

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']


- 목표 RNN : "Repeat is the best medicine for memory"을 입력 받으면 "is the best medicine for memory" 출력
- 먼저 임의의 문장으로부터 단어장 만들기

In [4]:
vocab = list(set(sentence))
print(vocab)

['for', 'the', 'best', 'Repeat', 'memory', 'medicine', 'is']


In [6]:
# 단어장의 단어에 고유한 정수 인덱스 부여
# 동시에 모르는 단어를 의미하는 unk 토큰 추가
word2index = {tkn : i for i, tkn in enumerate(vocab, 1)} # 1부터 시작
word2index['<unk>'] = 0
print(word2index)

{'for': 1, 'the': 2, 'best': 3, 'Repeat': 4, 'memory': 5, 'medicine': 6, 'is': 7, '<unk>': 0}


In [7]:
# 맵핑되는 정수 확인
print(word2index['memory'])

5


In [12]:
# 예측 단계에서 예측한 문장을 확인하기 위해 idx2word 만들기
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'the', 3: 'best', 4: 'Repeat', 5: 'memory', 6: 'medicine', 7: 'is', 0: '<unk>'}


In [18]:
print(index2word[3])

best


In [19]:
# 각 단어를 정수로 인코딩 & 입력 데이터와 레이블 데이터를 만드는 함수 만들기
def build_data(sentence, word2index) : 
    # 각 문자를 정수로 변환
    encoded = [word2index[token] for token in sentence]
    # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq, label_seq = encoded[:-1], encoded[1:]
    # 배치 차원 추가
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq

In [20]:
build_data(sentence, word2index)

(tensor([[4, 7, 2, 3, 6, 1]]), tensor([[7, 2, 3, 6, 1, 5]]))

In [25]:
X, Y = build_data(sentence, word2index)

In [26]:
print(X)
print(Y)

tensor([[4, 7, 2, 3, 6, 1]])
tensor([[7, 2, 3, 6, 1, 5]])


--------------------------------------------------------------------------------------------------------------

#### 2.모델 구현하기
- 이전 모델들과 달라진 점은 임베딩 층을 추가
- nn.Embedding()사용
    - 첫번째 인자 : 단어장의 크기
    - 두번째 인자 : 임베딩 벡터의 차원

In [32]:
class Net(nn.Module) : 
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True) :
        super(Net, self).__init__()
        # 워드 임베딩
        self.embedding_layer = nn.Embedding(num_embeddings = vocab_size,
                                            embedding_dim = input_size)
        # 입력 차원, 은닉 상태의 크기 정의
        self.rnn_layer = nn.RNN(input_size, hidden_size,
                                batch_first = batch_first)
        # 출력은 원-핫 벡터의 크기를 가져야함(또는 단어 집합의 크기만큼 가져야함)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x) : 
        # 1.임베딩 층
        # 크기변화 : (배치크기, 시퀀스 길이) -> (배치크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        
        # 2.RNN 층
        # 크기변화 : (배치크기, 시퀀스 길이, 임베딩 차원)
        # -> output(배치크기, 시퀀스 길이, 은닉층 크기),  hidden(1, 배치크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        
        # 3.최종 출력층
        # 크기변화 : ((배치크기, 시퀀스 길이, 은닉층 크기) -> (배치크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        
        # 4.view를 통해서 배치 차원 제거
        # 크기변화 : (배치크기, 시퀀스 길이, 단어장 크기) -> (배치크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [33]:
# 하이퍼 파라미터 설정
vocab_size = len(word2index)
input_size = 5
hidden_size = 20

In [34]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실함수 정의
loss_function = nn.CrossEntropyLoss()

# 옵티마이저 정의
optimizer = optim.Adam(params = model.parameters())

In [35]:
# 모델에 입력을 넣어서 출력 확인
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태
output = model(X)
print(output)

tensor([[ 0.0866, -0.1305, -0.1956,  0.0122,  0.0631,  0.0554,  0.1262, -0.0755],
        [-0.2107, -0.0824,  0.0587,  0.0621, -0.1442, -0.1231,  0.1131, -0.3176],
        [ 0.2743, -0.4139,  0.0957, -0.1013, -0.0333, -0.0693,  0.0624,  0.1643],
        [ 0.2008, -0.3720, -0.0981, -0.0104,  0.1004, -0.2586, -0.1132,  0.1261],
        [ 0.2625,  0.0938, -0.2225, -0.0737, -0.0582,  0.2045,  0.0414, -0.1901],
        [ 0.2627, -0.0088, -0.0251, -0.0085,  0.1476,  0.0650,  0.0800, -0.2306]],
       grad_fn=<ViewBackward0>)


In [36]:
# 현재 가중치는 랜덤 초기화되어 있어 의미있는 예측값은 아니다.
# 예측값의 크기 확인
print(output.shape)

torch.Size([6, 8])


- 예측값의 크기(6, 8) : 각각 (시퀀스의 길이, 단어장의 크기)에 해당
- 모델은 훈련시키기 전에 예측을 제대로 하고 있는지 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 **decode** 함수 만들기

In [37]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y : [index2word.get(x) for x in y]

In [40]:
# 훈련시작
for step in range(201) : 
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f}".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" " .join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0867
Repeat medicine medicine <unk> <unk> <unk> <unk>

[41/201] 1.5185
Repeat is the medicine medicine for memory

[81/201] 0.8788
Repeat is the best medicine for memory

[121/201] 0.4340
Repeat is the best medicine for memory

[161/201] 0.2270
Repeat is the best medicine for memory

[201/201] 0.1312
Repeat is the best medicine for memory

